## Let's use the Trapezoid Method, Simpson's Rule and Romberg Integration

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### Define a function for taking an integral

In [2]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

### Define it's integral so we know the right answer

In [3]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

## Define the core of the trapezoid method

In [4]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

### Define a wrapper function to perform trapezoid method

In [5]:
def trapezoid_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of functions
    #note; number of chunks will be N-1
    #so if the N is odd, then we don't need to adjust the
    #last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        #print(i,i+1,x[i]+h,x[-2],x[-1])
        
    #return the answer
    return Fint

### Define the core of the Simpson's Method

In [6]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3

### Define a wrapper function to perform Simpson's Method

In [7]:
def simpsons_method(f,a,b,N):
    #f == function to integrate
    #a == lower lim of integration
    #b == upper lim of integration
    #N == number of function evaluations to use
    #NOTE: the number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #def the value of the integral
    Fint = 0.0
    
    #perform the integral using the simpson's method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        
    #apply simpson's rule over the last interval
    #if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint

## Define the Romberg integration core

In [8]:
def romberg_core(f,a,b,i):
    
    #we need the difference b-a
    h = b-a
    
    #and the increment between new func evals
    dh = h/2.**(i)
    
    #we need a cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

## Define a wrapper function to perform Romberg Integration

In [9]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answer
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a)+ f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new fractional error
        delta = np.fabs( (I[i]-I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if (delta>tol):
            
            #iterate
            i+=1
            
            #check max iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
                
    #return the answer
    return I[i]

## Find out the answer

In [11]:
Answer = func_integral(1)-func_integral(0)
print("Answer")
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-6
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)

Answer
0.8866666666666665
Trapezoid
0.888744855967078
Simpson's Method
0.8866666666666665
Romberg
1 0.9603124999999997 1.0549999999999997 0.09860071591278888
2 0.9208593749999998 0.9603124999999997 0.04284381097819618
3 0.9031054687499998 0.9208593749999998 0.01965872964380725
4 0.8947216796874997 0.9031054687499998 0.009370275978367093
5 0.8906530761718747 0.8947216796874997 0.004568112573205605
6 0.8886495971679687 0.8906530761718747 0.0022545208035776375
7 0.8876555633544922 0.8886495971679687 0.0011198418108482326
8 0.8871604728698727 0.8876555633544922 0.000558061928771501
9 0.8869134092330915 0.8871604728698727 0.0002785656798162625
10 0.8867899978160845 0.8869134092330915 0.00013916645125782877
11 0.8867283222079236 0.8867899978160845 6.955412003449725e-05
12 0.8866974919289385 0.8867283222079236 3.476978255349039e-05
13 0.8866820786707006 0.8866974919289385 1.7383071800727052e-05
14 0.8866743725119841 0.8866820786707006 8.691080914686593e-06
15 0.8866705195500488 0.886674372511